In [2]:
just_checking_integrity=False

In [3]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [4]:
URM_test=sps.load_npz("../dataset/processed_data/URM_test_final.npz")

In [5]:
URM_test.shape

(4439967, 23690)

In [6]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [7]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [8]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [9]:
is_sorted(Session_ids)

True

In [10]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"]
for file in ["UCF","ICBF","Graph","ICF","UCF_W","ICBF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/final/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF
1523259
loaded UCF
0
loading ICBF
1485572
loaded ICBF
0
loading Graph
866733
loaded Graph
0
loading ICF
1815315
loaded ICF
0
loading UCF_W
1481491
loaded UCF_W
0
loading ICBF_W
1485572
loaded ICBF_W
0
loading Graph_W
910643
loaded Graph_W
0
loading ICF_W
1888294
loaded ICF_W
0
loading UCF_WW
1346714
loaded UCF_WW
0
loading ICBF_WW
1485572
loaded ICBF_WW
0
loading Graph_WW
910643
loaded Graph_WW
0
loading ICF_WW
1888294
loaded ICF_WW


In [11]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids


In [12]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [13]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [14]:
import ast
submission_df=pd.read_csv("../dataset/candidates/NN/GRU/final/candidates.csv")
submission_df["score"]=submission_df["score"].map(ast.literal_eval)
submission_df["code"]=submission_df["code"].map(ast.literal_eval)
submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
submission_df = submission_df.explode(['code','score', 'rank'])
submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [15]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/final/candidates.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [16]:
import ast
submission_df_LSTM=pd.read_csv("../dataset/candidates/NN/LSTM/final/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [17]:
dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [18]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [19]:
dataset.fillna(0,inplace=True)

In [20]:
gc.collect()

507

In [21]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN_final.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [22]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [23]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Score_UCF_W,Score_ICBF_W,Score_Graph_W,Score_ICF_W,Score_UCF_WW,...,Max_Score_ICF_WW,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.035168,0.115033,0.006554,0.015254,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19,0.018728,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23,0.038990,0.072503,0.007389,0.016038,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25,0.025525,0.081094,0.004554,0.008848,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,69,0.056398,0.085104,0.011284,0.016860,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
gc.collect()

23

In [25]:
dataset=dataset.sort_values("Session_Id")

In [26]:
gc.collect()

46

In [27]:
import lightgbm

In [28]:
k=10
result = []

In [29]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'../rankers/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))

result = np.array(result)
result

array([[-0.15051161, -0.15051161, -0.15051161, ..., -0.32809278,
        -0.32809278, -0.19812304],
       [-0.88355346, -0.88355346, -0.88355346, ..., -0.83473256,
        -0.83473256, -0.51231045],
       [-0.11524855, -0.11524855, -0.11524855, ..., -0.20596967,
        -0.20596967, -0.13619007],
       ...,
       [-1.12320008, -1.12320008, -1.12320008, ..., -1.28017068,
        -1.28017068, -0.94196749],
       [-2.39464736, -2.28751585, -2.28751585, ..., -2.72818723,
        -2.71762358, -1.39171856],
       [-2.32232676, -2.29119361, -2.29119361, ..., -2.24747093,
        -2.18862053, -1.50334854]])

In [30]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

In [31]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

130

In [32]:
dataset.head()

,Session_Id,Item_ID,score
2879128,61,11,-1.062496
4360668,61,1456,-1.046497
4360667,61,1446,-1.046497
4360666,61,1415,-1.046497
4360665,61,1413,-1.046497


In [33]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [34]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [35]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [36]:
dataset.iloc[0]

Session_Id                                                   61
Item_ID       [11, 1456, 1446, 1415, 1413, 1405, 1371, 1343,...
score         [-1.0624962095363641, -1.046497079485047, -1.0...
indices       [726, 706, 812, 740, 814, 596, 660, 636, 630, ...
Name: 0, dtype: object

In [37]:
dataset.iloc[0]["Item_ID"][536]

3344

In [38]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [39]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [40]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [41]:
dataset

,Session_Id,predictions,rank
0,61,"[1045, 292, 3018, 699, 2995, 3290, 1563, 3904,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[2346, 4587, 2370, 1608, 4433, 4347, 3550, 358...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4148, 2704, 4162, 4146, 1936, 3904, 535, 2643...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[2769, 16, 2259, 1689, 2057, 2077, 2089, 2546,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3599, 3346, 4477, 2533, 70, 1888, 1808, 503, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
9995,886222,"[4806, 143, 4574, 2995, 3290, 3586, 4083, 89, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
9996,886247,"[439, 2448, 70, 989, 3186, 2424, 3356, 16, 276...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
9997,886319,"[1516, 3026, 3871, 2170, 698, 3550, 1973, 1609...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
9998,886422,"[1876, 3863, 1671, 2593, 3237, 1634, 3184, 168...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [42]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [43]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [44]:
dataset.columns=["session_id","code","rank"]

In [45]:
dataset.head()

,session_id,code,rank
0,61,"(1045, 292, 3018, 699, 2995, 3290, 1563, 3904,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(2346, 4587, 2370, 1608, 4433, 4347, 3550, 358...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4148, 2704, 4162, 4146, 1936, 3904, 535, 2643...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(2769, 16, 2259, 1689, 2057, 2077, 2089, 2546,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3599, 3346, 4477, 2533, 70, 1888, 1808, 503, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [46]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [47]:
mapping= pd.read_csv('../dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [48]:
print(len(submission_df))

1000000


In [49]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   session_id  1000000 non-null  int32
 1   item_id     1000000 non-null  int64
 2   rank        1000000 non-null  int32
dtypes: int32(2), int64(1)
memory usage: 22.9 MB


In [50]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [51]:
submission_df.to_csv(
    f'../final_sub.csv',
    index=False
    )